In [3]:
import os
import simpy
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------
# 1. Queuing Simulation Functions (M/M/c/K)
# -------------------------------

class ChargingStation:
    def __init__(self, env, c, K, mu):
        """
        SimPy-based model of a charging station as an M/M/c/K queue.
        
        Parameters:
          env: SimPy environment.
          c: Number of chargers (servers).
          K: Total capacity (servers + waiting spaces).
          mu: Service rate (vehicles per minute).
        """
        self.env = env
        self.c = c
        self.K = K
        self.mu = mu
        self.servers = simpy.Resource(env, capacity=c)
        self.num_in_system = 0   # Count of vehicles (in service + waiting)
        self.blocked = 0         # Count of arrivals rejected due to full capacity
        self.total_arrivals = 0
        self.waiting_times = []  # Time from arrival until service start
        self.system_times = []   # Time from arrival until departure
        
        # To track the time spent in each state (n vehicles in system)
        self.state_time = {n: 0 for n in range(K+1)}
        self.last_event_time = 0.0

    def update_state(self, new_state):
        now = self.env.now
        dt = now - self.last_event_time
        prev_state = self.num_in_system
        if prev_state <= self.K:
            self.state_time[prev_state] += dt
        self.last_event_time = now
        self.num_in_system = new_state

def customer(env, station: ChargingStation):
    arrival_time = env.now
    station.total_arrivals += 1

    # Reject the arrival if the station is full.
    if station.num_in_system >= station.K:
        station.blocked += 1
        return

    # Vehicle enters the system.
    station.update_state(station.num_in_system + 1)

    # Request a charger.
    with station.servers.request() as req:
        yield req
        service_start = env.now
        waiting_time = service_start - arrival_time
        station.waiting_times.append(waiting_time)

        # Service time is exponentially distributed with rate mu.
        service_time = random.expovariate(station.mu)
        yield env.timeout(service_time)

        departure = env.now
        station.system_times.append(departure - arrival_time)
        station.update_state(station.num_in_system - 1)

def arrival_generator(env, station: ChargingStation, lam):
    """Generate arrivals according to a Poisson process with rate lam (vehicles per minute)."""
    while True:
        interarrival = random.expovariate(lam)
        yield env.timeout(interarrival)
        env.process(customer(env, station))

def run_simulation(lam, mu, c, K, sim_time):
    """
    Run the simulation for one station.
    
    Parameters:
      lam: Arrival rate (vehicles per minute).
      mu: Service rate (vehicles per minute).
      c: Number of chargers (servers).
      K: Total capacity (servers + waiting spaces).
      sim_time: Simulation time in minutes.
      
    Returns:
      A dictionary of simulation metrics.
    """
    random.seed(42)
    np.random.seed(42)
    
    env = simpy.Environment()
    station = ChargingStation(env, c, K, mu)
    env.process(arrival_generator(env, station, lam))
    env.run(until=sim_time)
    station.update_state(station.num_in_system)
    
    total_arrivals = station.total_arrivals
    blocked = station.blocked
    blocking_probability = blocked / total_arrivals if total_arrivals > 0 else 0
    avg_waiting_time = np.mean(station.waiting_times) if station.waiting_times else 0
    avg_system_time = np.mean(station.system_times) if station.system_times else 0
    total_time = sim_time
    avg_queue_length = sum(n * station.state_time[n] for n in station.state_time) / total_time
    
    # Approximate total service time (for utilization)
    total_service_time = sum([sys_time - wait for sys_time, wait in zip(station.system_times, station.waiting_times)])
    utilization = total_service_time / (c * sim_time) if sim_time > 0 else 0

    return {
        'blocking_probability': blocking_probability,
        'avg_waiting_time': avg_waiting_time,
        'avg_system_time': avg_system_time,
        'avg_queue_length': avg_queue_length,
        'utilization': utilization,
        'total_arrivals': total_arrivals,
        'blocked': blocked
    }

def mmck_theoretical(lam, mu, c, K):
    """
    Compute theoretical steady-state probabilities and performance metrics for an M/M/c/K queue.
    
    Returns a dictionary with:
      - 'P': List of steady-state probabilities P(n) for n = 0 to K.
      - 'P_loss': Blocking probability P(K).
      - 'q': Mean queue length.
      - 'w': Mean waiting time.
      - 'rho': Server utilization per server.
    """
    # Cast c and K to integers to use in range().
    c_int = int(c)
    K_int = int(K)
    
    sum1 = sum((lam/mu)**n / math.factorial(n) for n in range(c_int))
    sum2 = ((lam/mu)**c_int / math.factorial(c_int)) * sum((lam/(c_int*mu))**(n - c_int) for n in range(c_int, K_int+1))
    C = sum1 + sum2

    P = []
    for n in range(K_int+1):
        if n < c_int:
            Pn = ((lam/mu)**n / math.factorial(n)) / C
        else:
            Pn = ((lam/mu)**n / (math.factorial(c_int) * (c_int**(n-c_int)))) / C
        P.append(Pn)
    
    P_loss = P[K_int]
    q = sum((n - c_int) * P[n] for n in range(c_int, K_int+1))
    w = q / (lam * (1 - P_loss)) if lam * (1 - P_loss) > 0 else float('inf')
    rho = lam / (mu * c_int)
    
    return {
        'rho': rho,
        'P_loss': P_loss,
        'q': q,
        'w': w,
        'P': P
    }

# -------------------------------
# 2. Load Parameters from Phase 1 & 2
# -------------------------------

# File paths (please adjust if needed)
station_file = r'C:\Users\sayan\Downloads\EVCS\Data\Howard_Baltimore\Phase2_FinalCode\NSGAII\BMSA_1.61\0.5_SoC\0.9\charging_stations_locations.csv'
taz_file = r'C:\Users\sayan\Downloads\EVCS\Data\Howard_Baltimore\Phase2_FinalCode\NSGAII\BMSA_1.61\taz_data_unique_ev_trips.csv'

# Load station placement data from Phase 2.
df_stations = pd.read_csv(station_file)
# Expected columns include: 'Station', 'TAZ', 'Latitude', 'Longitude', 'Piles', 'Cost', 'Profit'

# Load TAZ-level EV trip data (from Phase 1/2).
df_taz = pd.read_csv(taz_file)
# Expected columns include: 'Origin_Lat', 'Origin_Lon', 'Trips', 'Proportion', 'EV_Trips'
# We assume each row corresponds to a unique TAZ and that the TAZ identifier matches df_stations['TAZ'].

# Global parameters from Phase 1:
T_day = 24 * 60  # Operating period: 1440 minutes (24 hours)
# The average charging time is derived from Phase 1 simulation and SoC considerations.
# For example, if you compute an average charging time of 30 minutes:
T_charge_eff = 30.0
mu = 1.0 / T_charge_eff  # Effective service rate (vehicles per minute)

# Define a design assumption for waiting capacity.
# For simplicity, assume waiting spaces equal the number of chargers, so K = 2*c.
def compute_capacity(c):
    return 2 * c

# -------------------------------
# 3. Run Queuing Analysis for Each Station
# -------------------------------

results_list = []

# Loop over each charging station from Phase 2.
for idx, station in df_stations.iterrows():
    taz_id = station['TAZ']
    # Retrieve the corresponding TAZ information.
    try:
        taz_info = df_taz.iloc[int(taz_id)]
    except Exception as e:
        print(f"Error finding TAZ {taz_id}: {e}")
        continue
    
    # Compute arrival rate (λ): assume EV_Trips is the total daily trips for that TAZ.
    EV_trips = taz_info['EV_Trips']
    lam = EV_trips / T_day  # vehicles per minute
    
    # Number of chargers (c) is taken from Phase 2 output (Piles); cast it to int.
    c = int(station['Piles'])
    # System capacity K: assume K = 2 * c.
    K = compute_capacity(c)
    
    # Compute theoretical metrics.
    theo_metrics = mmck_theoretical(lam, mu, c, K)
    
    # Run a discrete-event simulation over the operating period.
    sim_metrics = run_simulation(lam, mu, c, K, sim_time=T_day)
    
    results_list.append({
        'Station': station['Station'],
        'TAZ': taz_id,
        'Num_Chargers': c,
        'EV_Trips': EV_trips,
        'Arrival_Rate_lam': lam,
        'Capacity_K': K,
        'Service_Rate_mu': mu,
        'Theoretical_Blocking_Prob': theo_metrics['P_loss'],
        'Theoretical_Avg_Waiting_Time': theo_metrics['w'],
        'Theoretical_Mean_Queue_Length': theo_metrics['q'],
        'Theoretical_Utilization': theo_metrics['rho'],
        'Sim_Blocking_Prob': sim_metrics['blocking_probability'],
        'Sim_Avg_Waiting_Time': sim_metrics['avg_waiting_time'],
        'Sim_Mean_Queue_Length': sim_metrics['avg_queue_length'],
        'Sim_Utilization': sim_metrics['utilization']
    })

df_results = pd.DataFrame(results_list)
print("Integrated Queuing Metrics for Each Station:")
print(df_results.head())

# -------------------------------
# 4. Plotting Performance Metrics and Saving Outputs
# -------------------------------

# --- Define folder structure based on market penetration, SoC, and household percentage ---
base_output_folder = r'C:\Users\sayan\Downloads\EVCS\Data\Howard_Baltimore\Phase3_FinalCode'

# For example, for market penetration 1.61, SoC scenario 0.5, and household percentage 0.9:
market_penetration = "1.61"
soc_scenario = "0.5"
household_percentage = "0.9"

# Build the folder path and create folders if they don't exist.
output_folder = os.path.join(base_output_folder, market_penetration, soc_scenario, household_percentage)
os.makedirs(output_folder, exist_ok=True)

# Save the results table as CSV.
table_filepath = os.path.join(output_folder, "results_table.csv")
df_results.to_csv(table_filepath, index=False)
print(f"Results table saved to: {table_filepath}")

# Create a dictionary with units for each column.
units = {
    "Station": "ID (unitless)",
    "TAZ": "ID (unitless)",
    "Num_Chargers": "Number (unitless)",
    "EV_Trips": "Trips per day",
    "Arrival_Rate_lam": "vehicles/min",
    "Capacity_K": "Vehicles",
    "Service_Rate_mu": "vehicles/min",
    "Theoretical_Blocking_Prob": "Probability (unitless)",
    "Theoretical_Avg_Waiting_Time": "minutes",
    "Theoretical_Mean_Queue_Length": "Vehicles",
    "Theoretical_Utilization": "Fraction (unitless)",
    "Sim_Blocking_Prob": "Probability (unitless)",
    "Sim_Avg_Waiting_Time": "minutes",
    "Sim_Mean_Queue_Length": "Vehicles",
    "Sim_Utilization": "Fraction (unitless)"
}

# Save the units into a text file.
units_filepath = os.path.join(output_folder, "results_units.txt")
with open(units_filepath, "w") as f:
    for col, unit in units.items():
        f.write(f"{col}: {unit}\n")
print(f"Units information saved to: {units_filepath}")

# --- Original Plots ---

# Plot 1: Blocking Probability (Simulation vs. Theoretical)
plt.figure(figsize=(10,6))
plt.scatter(df_results['Station'], df_results['Sim_Blocking_Prob'], color='red', label='Simulated Blocking Probability')
plt.scatter(df_results['Station'], df_results['Theoretical_Blocking_Prob'], color='blue', label='Theoretical Blocking Probability')
plt.xlabel('Station ID')
plt.ylabel('Blocking Probability')
plt.title('Blocking Probability per Charging Station')
plt.legend()
plt.grid(True)
plot1_path = os.path.join(output_folder, "blocking_probability.png")
plt.savefig(plot1_path)
plt.close()
print(f"Plot saved to: {plot1_path}")

# Plot 2: Average Waiting Time (Simulation vs. Theoretical)
plt.figure(figsize=(10,6))
plt.scatter(df_results['Station'], df_results['Sim_Avg_Waiting_Time'], color='red', label='Simulated Avg Waiting Time')
plt.scatter(df_results['Station'], df_results['Theoretical_Avg_Waiting_Time'], color='blue', label='Theoretical Avg Waiting Time')
plt.xlabel('Station ID')
plt.ylabel('Average Waiting Time (minutes)')
plt.title('Average Waiting Time per Charging Station')
plt.legend()
plt.grid(True)
plot2_path = os.path.join(output_folder, "average_waiting_time.png")
plt.savefig(plot2_path)
plt.close()
print(f"Plot saved to: {plot2_path}")

# Plot 3: Mean Queue Length (Simulation) [Original]
plt.figure(figsize=(10,6))
plt.scatter(df_results['Station'], df_results['Sim_Mean_Queue_Length'], color='green', label='Simulated Mean Queue Length')
plt.xlabel('Station ID')
plt.ylabel('Mean Queue Length (vehicles)')
plt.title('Mean Queue Length per Charging Station (Simulation)')
plt.legend()
plt.grid(True)
plot3_path = os.path.join(output_folder, "sim_mean_queue_length.png")
plt.savefig(plot3_path)
plt.close()
print(f"Plot saved to: {plot3_path}")

# Plot 4: Utilization (Simulation vs. Theoretical)
plt.figure(figsize=(10,6))
plt.scatter(df_results['Station'], df_results['Sim_Utilization'], color='purple', label='Simulated Utilization')
plt.scatter(df_results['Station'], df_results['Theoretical_Utilization'], color='orange', label='Theoretical Utilization')
plt.xlabel('Station ID')
plt.ylabel('Utilization')
plt.title('Utilization per Charging Station')
plt.legend()
plt.grid(True)
plot4_path = os.path.join(output_folder, "utilization.png")
plt.savefig(plot4_path)
plt.close()
print(f"Plot saved to: {plot4_path}")

# --- Additional Plots with Rounded-Up "Vehicles" Values ---

# For columns with units "Vehicles": 
# "Sim_Mean_Queue_Length" and "Theoretical_Mean_Queue_Length"
# We'll generate additional plots with their values rounded up using np.ceil.

# Additional Plot A: Simulated Mean Queue Length (Rounded Up)
plt.figure(figsize=(10,6))
sim_mql_rounded = np.ceil(df_results['Sim_Mean_Queue_Length'])
plt.scatter(df_results['Station'], sim_mql_rounded, color='green', label='Simulated Mean Queue Length (Rounded Up)')
plt.xlabel('Station ID')
plt.ylabel('Mean Queue Length (vehicles, rounded up)')
plt.title('Simulated Mean Queue Length per Charging Station (Rounded Up)')
plt.legend()
plt.grid(True)
sim_mql_rounded_path = os.path.join(output_folder, "sim_mean_queue_length_rounded.png")
plt.savefig(sim_mql_rounded_path)
plt.close()
print(f"Plot saved to: {sim_mql_rounded_path}")

# Additional Plot B: Theoretical Mean Queue Length (Original vs. Rounded Up)
plt.figure(figsize=(10,6))
theo_mql_original = df_results['Theoretical_Mean_Queue_Length']
theo_mql_rounded = np.ceil(theo_mql_original)
plt.scatter(df_results['Station'], theo_mql_original, color='blue', label='Theoretical Mean Queue Length')
plt.scatter(df_results['Station'], theo_mql_rounded, color='orange', label='Theoretical Mean Queue Length (Rounded Up)', marker='x')
plt.xlabel('Station ID')
plt.ylabel('Mean Queue Length (vehicles)')
plt.title('Theoretical Mean Queue Length per Charging Station')
plt.legend()
plt.grid(True)
theo_mql_path = os.path.join(output_folder, "theoretical_mean_queue_length_combined.png")
plt.savefig(theo_mql_path)
plt.close()
print(f"Plot saved to: {theo_mql_path}")


Integrated Queuing Metrics for Each Station:
   Station  TAZ  Num_Chargers  EV_Trips  Arrival_Rate_lam  Capacity_K   
0      0.0  0.0             4     142.0          0.098611           8  \
1      1.0  1.0             4      68.0          0.047222           8   
2      2.0  4.0             4     189.0          0.131250           8   
3      3.0  5.0             4      50.0          0.034722           8   
4      4.0  7.0             4      71.0          0.049306           8   

   Service_Rate_mu  Theoretical_Blocking_Prob  Theoretical_Avg_Waiting_Time   
0         0.033333                   0.043025                      6.467170  \
1         0.033333                   0.000636                      0.686204   
2         0.033333                   0.131708                     11.741945   
3         0.033333                   0.000079                      0.232652   
4         0.033333                   0.000843                      0.795510   

   Theoretical_Mean_Queue_Length  Theoret